### This notebook is to just implement L1 and L2 regularization and compare with linear regression

#### Kaggle Dataset: https://www.kaggle.com/shivachandel/kc-house-data

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import train_test_split,cross_val_score
from statistics import mean

In [2]:
df=pd.read_csv('kc_house_data.csv')

In [3]:
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180.0,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170.0,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770.0,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050.0,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680.0,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530.0,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310.0,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020.0,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600.0,0,2004,0,98027,47.5345,-122.069,1410,1287


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21611 non-null  float64
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [5]:
df.drop(['id','date','zipcode'],axis=1,inplace=True)

In [6]:
df.isnull().sum()

price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       2
sqft_basement    0
yr_built         0
yr_renovated     0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [7]:
df.fillna(0,inplace=True)

In [8]:
Y=df.price
X=df.drop('price',axis=1)

In [9]:
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.3)

In [10]:
ytrain

7268     195000.0
18962    450000.0
8943     852600.0
19400    229950.0
14781    410000.0
           ...   
5796     699950.0
7136     222000.0
4150     260600.0
11012    308000.0
17924    420000.0
Name: price, Length: 15129, dtype: float64

#### Linear Regression

In [11]:
lr=LinearRegression()
lr.fit(xtrain,ytrain)

LinearRegression()

In [12]:
lr.score(xtest,ytest)

0.6958854455555058

#### Ridge regression

In [13]:
corss_val_scores_ridge=[]
alpha=[]

for i in range(1,9):
    ridgemodel=Ridge(alpha=i*0.25)
    alpha.append(i*0.25)
    ridgemodel.fit(xtrain,ytrain)
    scores=cross_val_score(ridgemodel,X,Y,cv=10)
    avg_cross_val_score=mean(scores)*100
    corss_val_scores_ridge.append(avg_cross_val_score)

for i in range(0,len(alpha)):
    print(str(alpha[i])+':'+str(corss_val_scores_ridge[i]))

0.25:69.0902113638978
0.5:69.09038805490131
0.75:69.09054500717272
1.0:69.09068242679562
1.25:69.0908005139155
1.5:69.09089946798987
1.75:69.090979483476
2.0:69.09104075454165


In [14]:
# we found that the alpha value with 2 has the best results 
ridgemodel=Ridge(alpha=2)
ridgemodel.fit(xtrain,ytrain)
ridgemodel.score(xtest,ytest)   

0.6957990652270283

#### Lasso Regression

In [17]:
corss_val_scores_lasso=[]
alpha=[]

for i in range(1,9):
    lassomodel=Lasso(alpha=i*0.25,tol=0.0925,normalize=True)
    alpha.append(i*0.25)
    lassomodel.fit(xtrain,ytrain)
    scores=cross_val_score(lassomodel,X,Y,cv=10)
    avg_cross_val_score=mean(scores)*100
    corss_val_scores_lasso.append(avg_cross_val_score)

for i in range(0,len(alpha)):
    print(str(alpha[i])+':'+str(corss_val_scores_lasso[i]))

0.25:69.08424038113056
0.5:69.08407320149249
0.75:69.08395723710939
1.0:69.0832456823066
1.25:69.0833691999011
1.5:69.08255193414425
1.75:69.08080154377923
2.0:69.08254479917201


In [19]:
lassomodel=Lasso(alpha=2,tol=0.0925)
lassomodel.fit(xtrain,ytrain)
lassomodel.score(xtest,ytest)   

0.6956820495406681

In [21]:
print("Linear Regression score:",lr.score(xtest,ytest)   )
print("Rigid  Regression score:",ridgemodel.score(xtest,ytest))
print("Lasso Regression score:",lassomodel.score(xtest,ytest))

Linear Regression score: 0.6958854455555058
Rigid  Regression score: 0.6957990652270283
Lasso Regression score: 0.6956820495406681
